# Make predictor for subsitutions into homology model

With 17 substitutions gives raise to more than 120000 potential variants.

In [1]:
import pandas as pd

In [2]:
!pwd

/Users/pgreisen/pythonscripts/rosetta_scripts/mutate_and_relax


In [3]:
# get mutations from fasta file which are going to be used as columns
aa_columns = []
number_of_variants = 0
with open("./files/tot.fasta",'r') as f:
    for i in f:
        if(i[0] == '>'):
            number_of_variants += 1
            tmpname = i[1:].strip().split('_')
            for j in tmpname[:-1]:
                aa_columns.append(j)

In [4]:
unique_aa = list(set(aa_columns))

In [5]:
df_rosetta_score = pd.read_csv("20190607_THCsynthetase_data/score_description.csv",delim_whitespace=True)

In [6]:
# add columns to dataframe
for i in unique_aa:
    df_rosetta_score[i] = 0

In [7]:
for i,j in zip(df_rosetta_score.index,df_rosetta_score["description"]):
    tmpmuts = j.split("_")
    for k in tmpmuts:
        df_rosetta_score.iloc[i,df_rosetta_score.columns.get_loc(k)] = 1

In [8]:
df_rosetta_score.to_csv("table_w_onehot_encoding.csv")

# Random Forest predictor

In [9]:
df_rosetta_score.describe()

,score,L326Y,A279Y,F223S,L325F,V342I,E376V,V225I,F271I,A374M,V338T,Q283I,R227S,I378Y,A407S,S229K,R340F,M405T
count,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000,45611.000000
mean,-1534.978085,0.477451,0.512135,0.488851,0.575760,0.474557,0.507400,0.414549,0.493828,0.495999,0.450527,0.485234,0.498235,0.474622,0.479029,0.450418,0.489093,0.482011
std,104.663996,0.499497,0.499858,0.499881,0.494232,0.499358,0.499951,0.492649,0.499967,0.499989,0.497552,0.499787,0.500002,0.499361,0.499566,0.497541,0.499886,0.499682
min,-1656.345000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-1601.193000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-1561.066000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,-1518.016500,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,-284.425000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import train_test_split

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

In [11]:
df_rosetta_score.head()

,score,description,L326Y,A279Y,F223S,L325F,V342I,E376V,V225I,F271I,A374M,V338T,Q283I,R227S,I378Y,A407S,S229K,R340F,M405T
0,-1004.611,L325F_A279Y_R227S_L326Y_V225I_V342I,1,1,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0
1,-1008.353,L325F_R227S_L326Y_Q283I_V338T_A279Y_R340F,1,1,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0
2,-1011.563,Q283I_A279Y_F223S_R227S_R340F_L326Y,1,1,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0
3,-1014.778,M405T_Q283I_L325F_L326Y_A279Y,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1
4,-1025.625,L325F_V225I_L326Y_E376V_V338T_A279Y_S229K,1,1,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0


In [12]:
X_aa_= df_rosetta_score[unique_aa]
Y_aa_= df_rosetta_score.iloc[:,df_rosetta_score.columns.get_loc("score")]

In [13]:
Y_aa_.to_csv("debug.csv")

In [14]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

scaler = preprocessing.MinMaxScaler()
scaled_results = scaler.fit_transform(Y_aa)

NameError: name 'Y_aa' is not defined

In [ ]:
print(scaled_results)
assert 1 == 0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_aa_,Y_aa_)

In [ ]:
# Fit regression model
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
clf.fit(X_train, y_train)
mse = mean_squared_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mse)

# #############################################################################
# Plot training deviance

# compute test set deviance
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, y_pred in enumerate(clf.staged_predict(X_test)):
    test_score[i] = clf.loss_(y_test, y_pred)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, clf.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')

# #############################################################################
# Plot feature importance
feature_importance = clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
# plt.yticks(pos, X_aa_.feature_names[sorted_idx])
# aa_pos
tmp = []
for i in sorted_idx:
    tmp.append(aa_pos[i])
    
plt.yticks(pos, tmp) # aa_pos[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()
plt.savefig("test.png")

# Gauss processes

In [ ]:
kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20,alpha=1e-6)
gp.fit(X_train, y_train)
print(gp.score(X_train, y_train))
print(gp.score(X_test, y_test))

In [ ]:
#scaler.inverse_transform(scaled_data)
